In this notebook, we convert a CellDesigner map into an executable qualitative model available as SBML-qual and GINsim files.

First, let's select a cell designer map.

In [1]:
# List xml files in the current folder
!ls *.xml

Apoptosis_03.06.2020.xml  IFN_26042020.xml


In [2]:
# Select the one we will use in this notebook (without the xml extension)
model_name = "IFN_26042020"
model_name = "Apoptosis_03.06.2020"

# Create an executable model with CaSQ¶


In [3]:
import casq.celldesigner2qual as casq

# debug messages are enabled by default
casq.logger.disable("casq")


# Load and simplify a cell designer map
info, width, height = casq.read_celldesigner(f"{model_name}.xml")
casq.simplify_model(info)

# Write the SBML file
casq.write_qual(f"{model_name}.sbml", info, width, height)

# SIF export for bioLQM

In [4]:
def biolqm_to_sif(m, sif_filename):
    matrix = biolqm.japi.java.jvm.org.colomoto.biolqm.ConnectivityMatrix(m)
    components = m.getComponents()
    out = open(sif_filename, 'w')
    for c in range(len(components)):
        rgs = matrix.getRegulators(c, False)
        efs = matrix.getRegulatorEffects(c, False)
        for (r, e) in zip(rgs, efs):
            for t in e:
                out.write(f'{components[r].getNodeID()} {t} {components[c].getNodeID()}\n')
    out.close()

# Load and view the model in GINsim¶

In [5]:
import biolqm
import ginsim


# Load the model in bioLQM
m = biolqm.load(f"{model_name}.sbml")
biolqm_to_sif(m, f"{model_name}_raw.sif")

# Transfer component names to identifiers
m = biolqm.sanitize(m)
biolqm_to_sif(m, f"{model_name}_sanitized.sif")

# Adjust the layout
layout = m.getLayout().scale(0.9, 0.9).crop()

# Transfer to GINsim
lrg = biolqm.to_ginsim(m)
ginsim.show(lrg)

In [6]:
# Save this model in GINsim format
lrg.save(f"{model_name}.zginml")